In [1]:
%%capture
!pip install datasets
!pip install contractions

In [2]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import re
from datasets import load_dataset
import contractions

## 1. Preprocessing

In [34]:
batch_size = 8
maxlen = 256
max_pred = 10

In [35]:
dataset = load_dataset("wikitext", 'wikitext-103-raw-v1')

In [36]:
for i in range(10):
  print(dataset['train'][i])

{'text': ''}
{'text': ' = Valkyria Chronicles III = \n'}
{'text': ''}
{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'}
{'text': " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adju

In [37]:
# tìm những dòng có text = ""
dataset_new = []
i = 0
len_train_data = len(dataset['train'])
while (i < 10000):
    if dataset['train'][i]['text'] == '':
        i+=3
        continue
    dataset_new.append(dataset['train'][i]['text'])
    i += 1
    print(i)

4
5
6
10
11
12
16
17
18
22
23
27
31
32
36
37
38
39
43
47
48
49
59
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
118
119
123
124
128
129
130
131
138
145
146
150
151
152
156
157
161
162
166
167
168
172
173
177
178
179
183
187
188
195
196
197
198
199
200
201
202
203
204
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
241
242
243
244
245
246
247
248
249
253
254
258
259
260
261
262
263
264
265
266
267
268
269
276
280
284
285
295
296
300
301
305
309
310
314
321
325
326
333
334
338
339
340
341
345
346
347
351
352
356
357
358
368
369
370
374
378
379
380
381
382
383
384
388
389
390
391
392
393
394
395
396
397
401
405
409
413
417
421
425
429
430
431
438
442
446
450
454
458
459
469
473
477
481
485
489
493
494
495
496
497
498
499
500
504
505
512
516
517
524
525
526
527
531
532
533
534
538
539
540
541
545

In [38]:
print(len(dataset_new))

4553


In [39]:
dataset_new[0]

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'

In [40]:
import re

def preprocess_text(text):
    text = contractions.fix(text)
    # Thay đổi dấu chấm trong dấu ngoặc tròn thành ký tự đặc biệt
    text = re.sub(r'\((.*?)\.(.*?)\)', r'(\1•\2)', text)
    # Chia câu dựa trên các dấu chấm (.), dấu phẩy (,), dấu chấm phẩy (;), dấu hai chấm (:), và ký tự cách (space)
    sentences = re.split(r'[.]', text.lower())
    # Loại bỏ các khoảng trắng thừa
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

# Đoạn văn bạn cung cấp
paragraph = "Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the ' Nameless ' , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit ' Calamaty Raven ' ."

# Tiền xử lý và chia câu
sentences = preprocess_text(paragraph)

# In ra các câu đã được chia
for idx, sentence in enumerate(sentences, start=1):
    print(f"Sentence {idx}: {sentence}")


Sentence 1: senjō no valkyria 3 : unrecorded chronicles ( japanese : 戦場のヴァルキュリア3 , lit • valkyria of the battlefield 3 ) , commonly referred to as valkyria chronicles iii outside japan , is a tactical role @-@ playing video game developed by sega and media
Sentence 2: vision for the playstation portable
Sentence 3: released in january 2011 in japan , it is the third game in the valkyria series
Sentence 4: employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the ' nameless ' , a penal military unit serving the nation of gallia during the second europan war who perform secret black operations and are pitted against the imperial unit ' calamaty raven '


In [41]:
# def preprocessing(text):
#   return re.sub("[,!?-]", '', text.lower()).split('.')  # filter '.', ',', '?', '!'
def preprocessing(text):
    text = contractions.fix(text)
    # Thay đổi dấu chấm trong dấu ngoặc tròn thành ký tự đặc biệt
    text = re.sub(r'\((.*?)\.(.*?)\)', r'(\1•\2)', text)
    # Chia câu dựa trên các dấu chấm (.), dấu phẩy (,), dấu chấm phẩy (;), dấu hai chấm (:), và ký tự cách (space)
    sentences = re.split(r'[.]', text.lower())
    # Loại bỏ các khoảng trắng thừa
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

In [42]:
dataset_sentences = []
dataset_word_list = []
for text in dataset_new:
    print(text)
    sentences = preprocessing(text)
    print(sentences)
    print(len(sentences))
    if len(sentences)<2:
        continue
    dataset_sentences.append(sentences)
    dataset_word_list.extend(list(set(" ".join(sentences).split())))

Output hidden; open in https://colab.research.google.com to view.

In [43]:
print(len(dataset_sentences))
print(dataset_word_list)

Output hidden; open in https://colab.research.google.com to view.

In [44]:
word_list = list(set(dataset_word_list))
print(len(word_list))

30808


In [45]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
   word_dict[w] = i + 4
number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)

In [46]:
print(word_dict)
print(vocab_size)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, 'minelaying': 4, 'box': 5, 'nowhere': 6, 'immunity': 7, 'maintain': 8, 'prompted': 9, 'subway': 10, 'minesweeper': 11, 'letting': 12, 'discredited': 13, 'to': 14, 'woke': 15, 'drawing': 16, 'claw': 17, 'degenerated': 18, 'yard': 19, 'gambit': 20, '166': 21, 'pincio': 22, 'rockwilder': 23, 'wharton': 24, 'foods': 25, 'milian': 26, 'cleaner': 27, 'raaf': 28, 'appealed': 29, 'rammed': 30, 'earthly': 31, 'avril': 32, 'greater': 33, 'stretching': 34, 'procured': 35, 'tv': 36, 'crest': 37, 'memphis': 38, '1233': 39, 'wider': 40, 'shawna': 41, 'alternates': 42, 'beleaguered': 43, 'penetrate': 44, 'raikes': 45, 'bone': 46, 'snow': 47, 'hall': 48, 'storeroom': 49, 'niño': 50, 'headwaters': 51, 'kettings': 52, 'community': 53, 'ssp': 54, 'horrific': 55, 'manipulates': 56, 'profitable': 57, 'fogong': 58, 'hornung': 59, 'venezuela': 60, 'thicker': 61, 'scary': 62, 'melora': 63, 'have': 64, 'ramblings': 65, 'distracts': 66, 'murchison': 67, 'striped

In [47]:
def token_list(text):
  return [word_dict[x] for x in text.split(' ') if x!='']

In [48]:
def text_list(tokens):
  tokens = tokens.tolist()
  print(tokens)
  return [number_dict.get(x, x) for x in tokens if x!=0]

In [49]:
len(dataset_sentences)

3592

In [50]:
def make_batch():
   batch = []
   positive = negative = 0
   sentences_index = random.randrange(len(dataset_sentences))
   sentences = dataset_sentences[sentences_index]
   i = 0
   while positive != batch_size/2 or negative != batch_size/2:
      #  print(i)
       i+=1
       tokens_a_index, tokens_b_index= random.randrange(len(sentences)), random.randrange(len(sentences))
      #  print(tokens_a_index, tokens_b_index)
      #  print(sentences[tokens_a_index],sentences[tokens_b_index])
       tokens_a, tokens_b= token_list(sentences[tokens_a_index].strip()), token_list(sentences[tokens_b_index].strip())

       input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
       segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

       # MASK LM
       n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % of tokens in one sentence
       cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
      #  print("input_ids", input_ids)
      #  print(cand_maked_pos)
       random.shuffle(cand_maked_pos)
      #  print(cand_maked_pos)
       masked_tokens, masked_pos = [], []
       for pos in cand_maked_pos[:n_pred]:
           masked_pos.append(pos)
           masked_tokens.append(input_ids[pos])
           if random.random() < 0.8:  # 80%
               input_ids[pos] = word_dict['[MASK]'] # make mask
           elif random.random() < 0.5:  # 10%
               index = random.randint(0, vocab_size - 1) # random index in vocabulary
               input_ids[pos] = word_dict[number_dict[index]] # replace
      #  print("masked_pos", masked_pos)
      #  print("masked_tokens", masked_tokens)
       # Zero Paddings
       n_pad = maxlen - len(input_ids)
       input_ids.extend([0] * n_pad)
       segment_ids.extend([0] * n_pad)

       # Zero Padding (100% - 15%) tokens
       if max_pred > n_pred:
           n_pad = max_pred - n_pred
           masked_tokens.extend([0] * n_pad)
           masked_pos.extend([0] * n_pad)

       if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
           positive += 1
       elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
           negative += 1
   return batch

In [51]:
batch = make_batch()
print(batch)

[[[1, 11022, 13634, 19486, 15491, 2, 11022, 13634, 3, 15491, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## 2. Embedding Layer

In [52]:
d_model = 512
n_segments = 2
n_heads = 4
d_k = d_v = d_model//n_heads
d_ff = 4*d_model
n_layers = 8

In [53]:
class Embedding(nn.Module):
   def __init__(self):
       super(Embedding, self).__init__()
       print("embedding: d_model = ", d_model)
       print("embedding: vocab_size = ", vocab_size)
       print("embedding: maxlen = ", maxlen)
       print("embedding: n_segments = ", n_segments)
       self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
       self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
       self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
       self.norm = nn.LayerNorm(d_model)

   def forward(self, x, seg):
       seq_len = x.size(1)
       pos = torch.arange(seq_len, dtype=torch.long)
       pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
       embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
       return self.norm(embedding)

## 3. Attention mask

In [54]:
def get_attn_pad_mask(seq_q, seq_k):
   batch_size, len_q = seq_q.size()
   batch_size, len_k = seq_k.size()
   # eq(zero) is PAD token
   pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
   return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

## 4. Scaled Dot Product Attention

In [55]:
class ScaledDotProductAttention(nn.Module):
   def __init__(self):
       super(ScaledDotProductAttention, self).__init__()

   def forward(self, Q, K, V, attn_mask):
       scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
       attn = nn.Softmax(dim=-1)(scores)
       context = torch.matmul(attn, V)
       return scores, context, attn

## 5. Multi Head Attention

In [56]:
class MultiHeadAttention(nn.Module):
   def __init__(self):
       super(MultiHeadAttention, self).__init__()
       print("MultiHeadAttention: d_model = ", d_model)
       print("MultiHeadAttention: d_k = ", d_k)
       print("MultiHeadAttention: d_v = ", d_v)
       self.W_Q = nn.Linear(d_model, d_k * n_heads)
       self.W_K = nn.Linear(d_model, d_k * n_heads)
       self.W_V = nn.Linear(d_model, d_v * n_heads)

   def forward(self, Q, K, V, attn_mask):
       # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
       residual, batch_size = Q, Q.size(0)
       # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
       q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
       k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
       v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
       attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]
       # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       _, context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
       context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
       output = nn.Linear(n_heads * d_v, d_model)(context)
       return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]

## 5. Position-Wise Feed Forward Network

In [57]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, dropout=0.1):
        super(PoswiseFeedForwardNet, self).__init__()
        print("PoswiseFeedForwardNet: d_model = ", d_model)
        print("PoswiseFeedForwardNet: dropout = ", dropout)
        print("PoswiseFeedForwardNet: d_ff = ", d_ff)
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## 6. Encoder

In [58]:
class EncoderLayer(nn.Module):
   def __init__(self):
       super(EncoderLayer, self).__init__()
       self.enc_self_attn = MultiHeadAttention()
       self.pos_ffn = PoswiseFeedForwardNet()

   def forward(self, enc_inputs, enc_self_attn_mask):
       enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
       enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
       return enc_outputs, attn

## 7. BERT

In [83]:
def gelu(x):
   return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [84]:
class BERT(nn.Module):
   def __init__(self):
       super(BERT, self).__init__()
       print("BERT: d_model = ",d_model)
       self.embedding = Embedding()
       self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
       self.fc = nn.Linear(d_model, d_model)
       self.activ1 = nn.Tanh()
       self.linear = nn.Linear(d_model, d_model)
       self.activ2 = gelu
       self.norm = nn.LayerNorm(d_model)
       self.classifier = nn.Linear(d_model, 2)
       # decoder is shared with embedding layer
       embed_weight = self.embedding.tok_embed.weight
       n_vocab, n_dim = embed_weight.size()
       self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
       self.decoder.weight = embed_weight
       self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

   def forward(self, input_ids, segment_ids, masked_pos):
       print(input_ids.device, segment_ids.device)
       output = self.embedding(input_ids, segment_ids)
       enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
       for layer in self.layers:
           output, enc_self_attn = layer(output, enc_self_attn_mask)
       # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
       # it will be decided by first token(CLS)
       h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
       logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

       masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]

       # get masked position from final output of transformer.
       h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
       h_masked = self.norm(self.activ2(self.linear(h_masked)))
       logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

       return logits_lm, logits_clsf

## 8. Loss and optimization

In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [86]:
model = BERT()

BERT: d_model =  512
embedding: d_model =  512
embedding: vocab_size =  30812
embedding: maxlen =  256
embedding: n_segments =  2
MultiHeadAttention: d_model =  512
MultiHeadAttention: d_k =  128
MultiHeadAttention: d_v =  128
PoswiseFeedForwardNet: d_model =  512
PoswiseFeedForwardNet: dropout =  0.1
PoswiseFeedForwardNet: d_ff =  2048
MultiHeadAttention: d_model =  512
MultiHeadAttention: d_k =  128
MultiHeadAttention: d_v =  128
PoswiseFeedForwardNet: d_model =  512
PoswiseFeedForwardNet: dropout =  0.1
PoswiseFeedForwardNet: d_ff =  2048
MultiHeadAttention: d_model =  512
MultiHeadAttention: d_k =  128
MultiHeadAttention: d_v =  128
PoswiseFeedForwardNet: d_model =  512
PoswiseFeedForwardNet: dropout =  0.1
PoswiseFeedForwardNet: d_ff =  2048
MultiHeadAttention: d_model =  512
MultiHeadAttention: d_k =  128
MultiHeadAttention: d_v =  128
PoswiseFeedForwardNet: d_model =  512
PoswiseFeedForwardNet: dropout =  0.1
PoswiseFeedForwardNet: d_ff =  2048
MultiHeadAttention: d_model =  512

In [87]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [88]:
model.to(device)

BERT(
  (embedding): Embedding(
    (tok_embed): Embedding(30812, 512)
    (pos_embed): Embedding(256, 512)
    (seg_embed): Embedding(2, 512)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (layers): ModuleList(
    (0-7): 8 x EncoderLayer(
      (enc_self_attn): MultiHeadAttention(
        (W_Q): Linear(in_features=512, out_features=512, bias=True)
        (W_K): Linear(in_features=512, out_features=512, bias=True)
        (W_V): Linear(in_features=512, out_features=512, bias=True)
      )
      (pos_ffn): PoswiseFeedForwardNet(
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (activ1): Tanh()
  (linear): Linear(in_features=512, out_features=512, bias=True)
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (classifier): L

In [90]:
print(f"Model device: {next(model.parameters()).device}")

Model device: cuda:0


In [89]:
for epoch in range(1000):
    optimizer.zero_grad()
    batch = make_batch()
    input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
    input_ids, segment_ids, masked_tokens, masked_pos, isNext = input_ids.to(device), segment_ids.to(device), masked_tokens.to(device), masked_pos.to(device), isNext.to(device)
    print(f"Input IDs device: {input_ids.device} {segment_ids.device} {masked_pos.device}")
    print(f"Model device: {next(model.parameters()).device}")
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)

    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss.item()))
    loss.backward()
    optimizer.step()

Input IDs device: cuda:0 cuda:0 cuda:0
Model device: cuda:0
cuda:0 cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# Predict mask tokens
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))

list1 = [number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]']
print(list1)
print([word_dict[i] for i in list1])


logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

# 9. Inference

In [ ]:
def make_input(input_string, pos = None):
    input_ids = token_list(input_string);
    print("input_ids", input_ids)
    sep1 = input_ids.index(2)
    sep2 = len(input_ids) - 1 - input_ids[::-1].index(2)
    segment_ids = [0] * (sep1+1) + [1] * (sep2 - sep1)

    if pos == None:
        masked_pos = [i for i, value in enumerate(input_ids) if value == 3]
    else:
        masked_pos = pos
    print("Các vị trí có giá trị là:", masked_pos)
    n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15))))
    if max_pred > n_pred:
        n_pad = max_pred - n_pred
        masked_pos.extend([0] * n_pad)
    return input_ids, segment_ids, masked_pos

In [ ]:
def predict(input_string, model, pos = None):
    input_ids, segment_ids, masked_pos = make_input(input_string, pos)
    # Chuyển đổi thành tensor
    input_ids = torch.LongTensor([input_ids])
    segment_ids = torch.LongTensor([segment_ids])
    masked_pos = torch.LongTensor([masked_pos])
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
    print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
    print('predict position masked tokens list :', logits_lm)
    print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

    logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
    print('isNext : ', True if isNext else False)
    print('predict isNext : ',True if logits_clsf else False)
    if isNext==logits_clsf:
        return "Model đã dự đoán đúng."
    else:
        return "Model đã dự đoán sai."

In [ ]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[3]))
print(input_ids, segment_ids, masked_tokens, masked_pos)
print(" ".join(text_list(input_ids[0])))

tensor([[ 1,  6, 24,  5, 18, 11,  3, 17,  2, 26, 18, 17,  2,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]) tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]) tensor([[11,  8,  0,  0,  0]]) tensor([[5, 6, 0, 0, 0]])
[1, 6, 24, 5, 18, 11, 3, 17, 2, 26, 18, 17, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] hello how are you i [MASK] romeo [SEP] thanks you romeo [SEP]


In [ ]:
string_input = "[CLS] hello how are you i [MASK] romeo [SEP] thanks you romeo [SEP]"
print(make_input(string_input, pos = [5, 6]))

input_ids [1, 6, 24, 5, 18, 11, 3, 17, 2, 26, 18, 17, 2]
Các vị trí có giá trị là: [5, 6]
([1, 6, 24, 5, 18, 11, 3, 17, 2, 26, 18, 17, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [5, 6, 0, 0, 0])


In [ ]:
print(masked_tokens)

tensor([[11,  8,  0,  0,  0]])


In [ ]:
print(predict(string_input, model, pos = [5, 6]))

input_ids [1, 6, 24, 5, 18, 11, 3, 17, 2, 26, 18, 17, 2]
Các vị trí có giá trị là: [5, 6]
masked tokens list :  [11, 8]
predict position masked tokens list : [11  8  0  0  0]
predict masked tokens list :  [11, 8]
isNext :  False
predict isNext :  True
Model đã dự đoán sai.


In [ ]:
number_dict[11]

'i'